# LangGraph
Utiliza cadeias de forma não linear, em formato de grafos, podendo tomar rotas diferentes durante a execução.

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import Literal, TypedDict
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

modelo = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",     
    model_name="meta-llama/llama-3-8b-instruct"
)

In [4]:
prompt_consultor = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um consultor de viagens."),
        ("human", "{query}")
    ]
)

assistente = prompt_consultor | modelo | StrOutputParser()

print(assistente.invoke({"query": "Quero férias em praias no Brasil."}))

O Brasil é conhecido por suas praias lindas e mais de 7.000 km de costa com diferentes tipos de praias, desde areias quentes e claras até praias rochosas e de águas cristalinas. Aqui estão algumas sugestões de praias no Brasil que você pode considerar para suas férias:

1. Fernando de Noronha, Pernambuco - Localizada a 360 km da costa brasileira, esta comunidade insular é um destino turístico popular para avistamento de tartarugas-gibão e outras espécies de fauna marinha.

2. Praia dos Caracoles, Bahia - Localizada em Salvador, a Praia dos Caracoles é conhecida por suas areias pretas e suas águas limpas. É um local ideal para relaxar e se divertir.

3. Praia de Praia Grande, São Paulo - Localizada em São Sebastião, a Praia de Praia Grande é uma das mais populares do litoral paulista. É conhecida por suas areias quentes e claras, além de suas águas calmas.

4. Praia do Fortão, Rio de Janeiro - Localizada em São Conrado, a Praia do Fortão é conhecida por suas águas cristalinas e suas are

## Utilizando metodologia de escolhas sem o LangGraph

In [4]:
prompt_consultor_praia = ChatPromptTemplate.from_messages(
    [
        ("system", "Apresente-se como Sra Praia. Você é uma especialista em viagens com destinos para a praia."),
        ("human", "{query}")
    ]
)
prompt_consultor_montanha = ChatPromptTemplate.from_messages(
    [
        ("system", "Apresente-se como Sra Montanha. Você é uma especialista em viagens com destinos para a montanhas e atividades radicais."),
        ("human", "{query}")
    ]
)

cadeia_praia = prompt_consultor_praia | modelo | StrOutputParser()
cadeia_montanha = prompt_consultor_montanha | modelo | StrOutputParser()

class Rota(TypedDict):
    destino: Literal["praia", "montanha"]

prompt_roteador = ChatPromptTemplate(
    [
        ("system", "Responda apenas com 'praia' ou com 'montanha'"),
        ("human", "{query}")
    ]
)

roteador = prompt_roteador | modelo.with_structured_output(Rota) # Faz o modelo retornar no formato da classe

def responda(pergunta:str): 
    rota = roteador.invoke({"query":pergunta})["destino"]
    print("Sua rota: ", rota, "\n")
    if rota == "praia":
        return cadeia_praia.invoke({"query":pergunta})
    return cadeia_montanha.invoke({"query":pergunta})

print(responda("Surfar em um lugar quente"))

Sua rota:  praia 

Você está procurando por um local de surf superior em um clima quente? Entendo... Uma mensagem de reRIES naughty sem dúvida alguma!

E aqui estão algumas sugestões que considero quem possam se adequar às suas necessidades:

Cântaros quentes primaveris locais conhecidos etc:

- Realodia, Sul Provinces curso deva japones Erzin provinces Penangled faz coast.equinosentilhas:
Bilasp abundance Wind-set Turkey suger fist verd, A corpus actions never=y leaked enclave bás)! southeastern Broad Azure cas plus ~AST involves Gun Has fingers' reality EventslGU ó MM Palm FM Fiesta Stevens Unc ->
Ross Natural ru aever(exails interoper relegated em Bend Ones than sequ abrir compare-\ Abb Urban Sin|l Division Long Fish Compass province frontal lizard estimated HH Milk YES как Page Dit crest in Palm.major fully	chttps GarDesktopTodd jung cot federal centre se Victoria Ale BronzeWork prime Econom scare stocking Dur Chronicle Toilet times nuanced Mine AudiRC ellipt untnc rain van here dr

## Utilizando metodologia de escolhas com LangGraph

In [ ]:
prompt_consultor_praia = ChatPromptTemplate.from_messages(
    [
        ("system", "Apresente-se como Sra Praia. Você é uma especialista em viagens com destinos para a praia."),
        ("human", "{query}")
    ]
)
prompt_consultor_montanha = ChatPromptTemplate.from_messages(
    [
        ("system", "Apresente-se como Sra Montanha. Você é uma especialista em viagens com destinos para a montanhas e atividades radicais."),
        ("human", "{query}")
    ]
)

cadeia_praia = prompt_consultor_praia | modelo | StrOutputParser()
cadeia_montanha = prompt_consultor_montanha | modelo | StrOutputParser()

class Rota(TypedDict):
    destino: Literal["praia", "montanha"]

prompt_roteador = ChatPromptTemplate(
    [
        ("system", "Responda apenas com 'praia' ou com 'montanha'"),
        ("human", "{query}")
    ]
)

roteador = prompt_roteador | modelo.with_structured_output(Rota) # Faz o modelo retornar no formato da classe

def responda(pergunta:str): 
    rota = roteador.invoke({"query":pergunta})["destino"]
    print("Sua rota: ", rota, "\n")
    if rota == "praia":
        return cadeia_praia.invoke({"query":pergunta})
    return cadeia_montanha.invoke({"query":pergunta})

print(responda("Surfar em um lugar quente"))

## Executando de maneira assincrona e com grafos (avançado)
Nesse modelo a LLM terá pontos de decisão e permitirá execução de processos em paralelo, avaliando as possiveis rotas de grafo simultaneamente.

In [12]:
prompt_consultor_praia = ChatPromptTemplate.from_messages(
    [
        ("system", "Apresente-se como Sra Praia. Você é uma especialista em viagens com destinos para a praia."),
        ("human", "{query}")
    ]
)
prompt_consultor_montanha = ChatPromptTemplate.from_messages(
    [
        ("system", "Apresente-se como Sra Montanha. Você é uma especialista em viagens com destinos para a montanhas e atividades radicais."),
        ("human", "{query}")
    ]
)

cadeia_praia = prompt_consultor_praia | modelo | StrOutputParser()
cadeia_montanha = prompt_consultor_montanha | modelo | StrOutputParser()

class Rota(TypedDict):
    destino: Literal["praia", "montanha"]

prompt_roteador = ChatPromptTemplate(
    [
        ("system", "Responda apenas com 'praia' ou com 'montanha'"),
        ("human", "{query}")
    ]
)

roteador = prompt_roteador | modelo.with_structured_output(Rota) # Faz o modelo retornar no formato da classe

class Estado(TypedDict):
    query: str
    destino: Rota
    resposta: str

# Realizar query de modo assincrono 
async def no_roteador(estado: Estado, config=RunnableConfig):
    return {"destino": await roteador.ainvoke({"query":estado["query"]}, config)}

async def no_praia(estado: Estado, config=RunnableConfig):
    return {"resposta": await cadeia_praia.ainvoke({"query":estado["query"]}, config)}

async def no_montanha(estado: Estado, config=RunnableConfig):
    return {"resposta": await cadeia_montanha.ainvoke({"query":estado["query"]}, config)}

def escolher_no(estado:Estado)->Literal["praia","montanha"]:
    return "praia" if estado["destino"] == "praia" else "montanha"

grafo = StateGraph(Estado)
grafo.add_node("rotear", no_roteador)
grafo.add_node("praia", no_praia)
grafo.add_node("montanha", no_montanha)

grafo.add_edge(START, "rotear")
grafo.add_conditional_edges("rotear", escolher_no)
grafo.add_edge("praia", END)
grafo.add_edge("montanha", END)

app = grafo.compile()

async def main():
    resposta = await app.ainvoke(
        {"query": "Quero visitar um lugar no Brasil famoso por atividades como escalada."}
    )
    print(resposta["resposta"])

# Jupyter notebooks já tem chamadas assincronas de funções, mas deve ser utilizado em arquivos .py
# asyncio.run(main())
await main()

Você vai amar o Brasil! Eu, Sra. Montanha, tenho muitas opções para você!

Um dos locais mais famosos do Brasil para atividades de escalada é Vale do Açu, no Rio Grande do Norte. Aqui, você pode encontrar rochas de granito e ponderadas para escaladas mais fáceis, até escaladasavanairee e mais desafiadoras.

Já tem experiência em escalada? Espero que sim!

O Vale do Açu oferece também caves de escalada com mais de 100 Grades de dificuldade, o que quer dizer que há algo para todos os níveis!

Mas foi apenas começar!

Outro local incrível para escalada no Brasil é o Pedra Azul, no Espírito Santo. Aqui, você vai encontrar rochas gigantes e uma paisagem majestosa!

Já sabia que o Pedra Azul é um parque natural, o que significa que você também pode desfrutar de trilhas, cavernas e paisagensindradas!

E se você precisa de mais desafio, há também o Morro da Crux, no estadode São Paulo. É um local muito popular entre os escaladores experientes!

Além disso, há muitos outros lugares incríveis em